In [5]:
API_URL = "http://127.0.0.1:8000/translate"  # đổi nếu server/ngrok/domain khác
HEALTH_URL = "http://127.0.0.1:8000/health"


In [6]:
import requests
r = requests.get(HEALTH_URL, timeout=10)
print(r.status_code, r.json())


200 {'status': 'ok', 'model': 'models/gemini-2.0-flash'}


In [7]:
import time, requests

def call_translate(text, source="auto", target="vi", style="neutral", glossary=None,
                   api_url=API_URL, timeout=60, retries=2, backoff=1.5):
    payload = {
        "text": text,
        "source": source,
        "target": target,
        "style": style,
        "glossary": glossary or None
    }
    last_err = None
    for i in range(retries + 1):
        try:
            r = requests.post(api_url, json=payload, timeout=timeout)
            if r.ok:
                return r.json()["translation"]
            else:
                try:
                    detail = r.json().get("detail")
                except Exception:
                    detail = r.text
                raise RuntimeError(f"HTTP {r.status_code}: {detail}")
        except Exception as e:
            last_err = e
            if i < retries:
                time.sleep(backoff ** i)
            else:
                raise last_err


In [ ]:
print(
    call_translate(
        ".",
        source="vi", target="en",
        glossary={"ship": "giao hàng"}
    )
)


Friday.


In [ ]:
texts = [
    "Setup usually takes around 5 minutes.",
    "Your account has been suspended due to unusual activity.",
    "Click the button below to continue."
]
results = [call_translate(t, "en", "vi") for t in texts]

import pandas as pd
pd.DataFrame({"source_en": texts, "target_vi": results})


Lá thu rơi nhẹ nhàng,
Gió heo may khẽ hát vang.

